# Imports

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive,spark
from wmfdata.utils import print_err, pd_display_all
from wmfdata.utils import check_kerberos_auth, ensure_list


You are using wmfdata v1.0.3, but v1.0.4 is available.

To update, run `pip install --upgrade git+https://github.com/neilpquinn/wmfdata.git@release`.

To see the changes, refer to https://github.com/neilpquinn/wmfdata/blob/release/CHANGELOG.md


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
# METRICS_MONTH_TEXT = '2019-07'
# MEDIAWIKI_HISTORY_SNAPSHOT = '2019-07'

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")

# Preparation

In [3]:

# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = pd.Period(METRICS_MONTH_TEXT)
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    "metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str(metrics_month.asfreq("D", how="start")),
    "metrics_month_end": str((metrics_month + 1).start_time),
    "metrics_month_last_day": str(metrics_month.asfreq("D", how="end")),
    "api_metrics_month_first_day": metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    "api_metrics_month_day_after": (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    "metrics_prev_month": str(metrics_month - 1),
    "retention_cohort": str(metrics_month - 2)
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

In [4]:
date_params

{'mediawiki_history_snapshot': '2020-07',
 'metrics_month': '2020-07',
 'metrics_month_start': '2020-07-01 00:00:00',
 'metrics_month_first_day': '2020-07-01',
 'metrics_month_end': '2020-08-01 00:00:00',
 'metrics_month_last_day': '2020-07-31',
 'api_metrics_month_first_day': '20200701',
 'api_metrics_month_day_after': '20200801',
 'metrics_prev_month': '2020-06',
 'retention_cohort': '2020-05'}

# MariaDB and Hive query metrics

In [6]:
queries = {
    "active_editors": {
        "file": "queries/active_editors.sql",
        "engine": "hive"
    },
    "edits": {
        "file": "queries/edits.hql",
        "engine": "hive"
    },
    "new_editor_retention": {
        "file": "queries/new_editor_retention.hql",
         "engine": "hive"
    },
    "global_south_edits_editors": {
        "file": "queries/global_south_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_edits_editors": {
        "file": "queries/mobile-heavy_edits_editors.hql",
        "engine": "hive"
    },
    "mobile-heavy_new_editor_retention": {
        "file": "queries/mobile-heavy_new_editor_retention.hql",
        "engine": "hive"
    },
    "global_south_new_editor_retention": {
        "file": "queries/global_south_new_editor_retention.hql",
        "engine": "hive"
    }
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running active_editors on hive...
Running edits on hive...
Running new_editor_retention on hive...
Running global_south_edits_editors on hive...
Running mobile-heavy_edits_editors on hive...
Running mobile-heavy_new_editor_retention on hive...
Running global_south_new_editor_retention on hive...


# Content metrics via API

In [4]:
NEW_PAGES_API = (
    # Replaces "https://wikimedia.org/api/rest_v1/metrics/" due to https://phabricator.wikimedia.org/P8605
    "http://aqs1004.eqiad.wmnet:7232/analytics.wikimedia.org/v1/" 
    "edited-pages/new/{project}/all-editor-types/{page_type}/monthly/{start}/{end}"
)

headers = {
    "User-Agent": "https://github.com/wikimedia-research/Editing-movement-metrics (bot)"
}

# Create container for results
api_results = []

def get_new_pages(
    project="all-projects",
    page_type="content",
    start= date_params["api_metrics_month_first_day"],
    end= date_params["api_metrics_month_day_after"]
):
    url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )
    
    r = requests.get(url, headers=headers)
    data = r.json()["items"][0]["results"]
    frame = pd.DataFrame(data)
    frame["timestamp"] = pd.to_datetime(frame["timestamp"]).dt.tz_localize(None)
    frame = frame.rename(columns={"timestamp": "month"})
    
    return frame

In [5]:
project="all-projects",
page_type="content"
start=date_params["api_metrics_month_first_day"]
end=date_params["api_metrics_month_day_after"]

In [6]:
 url = NEW_PAGES_API.format(
        project = project,
        page_type = page_type,
        start = start,
        end = end
    )

## Total

In [11]:
total_new = get_new_pages().rename(columns={"new_pages": "net_new_content_pages"})
api_results.append(total_new)

## Wikidata

In [12]:
new_wd = get_new_pages(
    project="wikidata.org"
).rename(columns={
    "new_pages": "net_new_Wikidata_entities"
})
api_results.append(new_wd)

## Commons

In [13]:
new_commons = get_new_pages(
    project="commons.wikimedia.org"
).rename(columns={
    "new_pages": "net_new_Commons_content_pages"
})
api_results.append(new_commons)

## Wikipedias

In [14]:
# Get a list of project URLs (each one in a 1-tuple)
wp_domains = spark.run("""
select domain_name
from canonical_data.wikis
where database_group = "wikipedia"
""", format="raw")

# Query the API for each project and append records to a list
results = []
n = len(wp_domains)

In [15]:

for idx, val in enumerate(wp_domains):
    domain = val[0]
    
    if idx % 10 == 0:
        msg = "Now on the {}th project of {} ({})"
        print_err(msg.format(idx, n, domain))
        
    frame = get_new_pages(project=domain).reset_index()
    frame["project"] = domain
    records = frame.to_dict("records")
    results.extend(records)
    
    # Sleep 20 milliseconds
    time.sleep(0.02)

# Turn the big list of records into a data frame
new_per_wp = pd.DataFrame(results)

# Sum across projects to get new Wikipedia articles per month
new_wp = new_per_wp.groupby("month").agg(
    {"new_pages": "sum"}
).rename(columns={"new_pages": "net_new_Wikipedia_articles"}).reset_index()

api_results.append(new_wp)

Now on the 0th project of 308 (aa.wikipedia.org)
Now on the 10th project of 308 (arc.wikipedia.org)
Now on the 20th project of 308 (ban.wikipedia.org)
Now on the 30th project of 308 (bjn.wikipedia.org)
Now on the 40th project of 308 (cbk-zam.wikipedia.org)
Now on the 50th project of 308 (crh.wikipedia.org)
Now on the 60th project of 308 (diq.wikipedia.org)
Now on the 70th project of 308 (et.wikipedia.org)
Now on the 80th project of 308 (frr.wikipedia.org)
Now on the 90th project of 308 (gl.wikipedia.org)
Now on the 100th project of 308 (he.wikipedia.org)
Now on the 110th project of 308 (hz.wikipedia.org)
Now on the 120th project of 308 (is.wikipedia.org)
Now on the 130th project of 308 (kbd.wikipedia.org)
Now on the 140th project of 308 (ko.wikipedia.org)
Now on the 150th project of 308 (la.wikipedia.org)
Now on the 160th project of 308 (lo.wikipedia.org)
Now on the 170th project of 308 (mh.wikipedia.org)
Now on the 180th project of 308 (mt.wikipedia.org)
Now on the 190th project of 30

In [16]:
# Strip timezones returned by API so our month columns merge nicely
for df in api_results:
    df["month"] = df["month"].dt.tz_localize(None)

# Combining and saving metrics

In [17]:
queries["edits"]["result"] = queries["edits"]["result"].rename({"data_edits": "wikidata_edits"}, axis=1)

In [18]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]
results.extend(api_results)

In [19]:
# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail())

,active_editors,anonymous_edits,global_south_active_editors,global_south_edits,global_south_new_editor_retention,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,mobile_edits,net_new_Commons_content_pages,net_new_Wikidata_entities,net_new_Wikipedia_articles,net_new_content_pages,new_active_editors,new_editor_retention,non_anonymous_edits,other_nonbot_edits,returning_active_editors,revert_rate,total_edits,uploads,wikidata_edits
month,,,,,,,,,,,,,,,,,,,,,,,,
2020-03-01,90443.0,2222674.0,21524.0,2802052.0,0.042328,2441182.0,5028.0,2955059.0,0.046599,702907.0,1678852.0,933836.0,2368419.0,468599.0,4041475.0,18254.0,0.071413,47670116.0,13262731.0,72189.0,0.054138,49822981.0,998024.0,20754272.0
2020-04-01,95994.0,2221239.0,23011.0,3374326.0,0.041195,2762677.0,5534.0,2023867.0,0.046041,825048.0,1741987.0,505900.0,3418000.0,491300.0,4720000.0,20606.0,0.063241,50802899.0,15206640.0,75388.0,0.053670,53021129.0,567161.0,22620119.0
2020-05-01,104508.0,2365101.0,25316.0,4189735.0,0.049743,2998223.0,6001.0,2392095.0,0.054792,778171.0,1924709.0,878203.0,2929482.0,396374.0,4512879.0,24774.0,0.069265,52383642.0,14663802.0,79734.0,0.060618,54751101.0,937664.0,25043268.0
2020-06-01,96014.0,2145261.0,23312.0,3417381.0,0.051572,2516055.0,5321.0,2117588.0,0.048071,813305.0,1791421.0,649618.0,511455.0,487908.0,1922896.0,21334.0,0.067197,50814018.0,12869134.0,74680.0,0.057797,52960770.0,703996.0,25836962.0
2020-07-01,87691.0,2087988.0,23467.0,3133019.0,0.042145,2721998.0,5356.0,1496188.0,0.045204,799567.0,1740901.0,743741.0,913741.0,387281.0,2282643.0,18660.0,0.055230,44143758.0,11996820.0,69031.0,0.066772,46233269.0,801286.0,21846294.0


In [20]:
results

[       month  active_editors  new_active_editors  returning_active_editors
 0 2020-07-01           87691               18660                     69031,
        month  total_edits  uploads  mobile_edits  wikidata_edits  \
 0 2020-07-01     46233269   801286       1740901        21846294   
 
    other_nonbot_edits  anonymous_edits  non_anonymous_edits  revert_rate  
 0            11996820          2087988             44143758     0.066772  ,
        month  new_editor_retention
 0 2020-07-01               0.05523,
        month  global_south_edits  global_south_nonbot_edits  \
 0 2020-07-01             3133019                    2721998   
 
    global_south_active_editors  
 0                        23467  ,
        month  mobile-heavy_wiki_edits  mobile-heavy_wiki_nonbot_edits  \
 0 2020-07-01                  1496188                          799567   
 
    mobile-heavy_wiki_active_editors  
 0                              5356  ,
        month  mobile-heavy_wiki_new_editor_retention

In [21]:
metrics.to_csv(FILENAME, sep="\t")